The following is the default initialisation code. Use (?) it from another notebook using

```
%run "! DEFAULT PLOT THINGIES.ipynb"
%run "../! DEFAULT PLOT THINGIES.ipynb"
```

and all should be 'swell. (the double run is necessary for after the chdir)

In [4]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "theNotebook = " + "'"+thename+"'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [5]:
import os, socket, IPython

basedir = {"MEETPC116": "c:/users/deen/dropbox/phd rearrangements/thesis/notebooks"}[socket.gethostname()]
try:
    path = os.path.join(basedir, theNotebook)
    
    if "_" in theNotebook:
        raise Exception("Notebook name must not contain underscores, or LaTeX will complain!")

    if not theNotebook.startswith("!"):
        try:
            os.makedirs(path)
        except OSError:
            pass

        os.chdir(path)
        print "Current working directory is now", os.getcwd()
except NameError:
    print "Please re-run this cell (should be automagic, but what can one do...)"
    
    IPython.core.display.display_javascript(""" 
    var c = IPython.notebook.get_cell(0);
    if (c.element[0].innerHTML.substr("! DEFAULT PLOT THINGIES.ipynb") != 1) {
        c.execute();
    }""", raw=True)

In [6]:
%pylab inline
from __future__ import division
import glob
import sys, os
import pandas
import pandas as pd

fig_width_pt = 355 #pt; ~165 mm
inches_per_pt = 1.0/72.27               # Convert pt to inches
golden_mean = (sqrt(5)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
fig_height =fig_width/1.5     # height in inches

fig_size = [fig_width,fig_width/2.05] #fig_height]
half_fig_size = [fig_width/2.05, fig_width/2.05]


    


fig_sppars_dict = dict(left = 0.125, bottom=0.19, right=0.97, top=0.94)
fig_sppars = mpl.figure.SubplotParams(**fig_sppars_dict)

half_fig_sppars_dict = dict(left = fig_sppars.left*2,
                            bottom = fig_sppars.bottom,
                            right = 1-(1-fig_sppars.right)*2,
                            top = fig_sppars.top)
half_fig_sppars = mpl.figure.SubplotParams(**half_fig_sppars_dict)

fpf_width_pt = 512
fpf_height_pt = 355 - 20 # 20 pt for caption
fpf_width = fpf_width_pt * inches_per_pt
fpf_height = fpf_height_pt * inches_per_pt
fpf_size = [fpf_width, fpf_height]

fpf_sppars_dict = dict(
    left = fig_sppars.left * fig_width / fpf_width,
    bottom = fig_sppars.bottom * fig_height / fpf_height,
    right = 1 - (1 - fig_sppars.right) * fig_width / fpf_width,
    top = 1 - (1 - fig_sppars.top) * fig_height / fpf_height
)

fpf_sppars = mpl.figure.SubplotParams(**fpf_sppars_dict)

def reset_subplots_adjust():
    fig = plt.gcf()
    if fig.subplotpars == fig_sppars:
        subplots_adjust(**fig_sppars_dict)
    elif fig.subplotpars == half_fig_sppars:
        subplots_adjust(**half_fig_sppars_dict)
    elif fig.subplotpars == fpf_sppars:
        subplots_adjust(**fpf_sppars)
    else:
        raise Exception('unknown subplotparams!')

    
rc('text.latex', preamble = r"\usepackage{mathpazo}\usepackage{nicefrac}")
rc('text', usetex=True)
rc('figure', figsize=fig_size)
rc('font', size=10)
rc('font', family='serif')
rc('savefig', dpi=300)

rc('image', cmap='viridis')


def fullfig(**kwargs):
    params = {'figsize': fig_size[:], 'subplotpars': fig_sppars}
    params.update(kwargs)
    if 'hf' in params:
        params['figsize'][1] *= params.pop('hf')
    return figure(**params)

def halffig(**kwargs):
    params = {'figsize': half_fig_size[:], 'subplotpars': half_fig_sppars}
    params.update(kwargs)
    if 'hf' in params:
        params['figsize'][1] *= params.pop('hf')
    return figure(**params)

def pagefig(**kwargs):
    params = {'figsize': fpf_size[:], 'subplotpars': fpf_sppars}
    params.update(kwargs)
    if 'hf' in params:
        params['figsize'][1] *= params.pop('hf')
    return figure(**params)    

def subplotlabel(label, loc='upper left', offsetx=6, offsety=6, ax=None, **kwargs):
    """ e.g. subplotlabel('(a)', 'upper right')
    upper/lower, left/right are accepted
    offset is in bbox width units
    """

    if ax is None:
        ax = gca()
    
    v,h = loc.split(' ')
    
    axh = ax.bbox.height
    axw = ax.bbox.width
    
    if v == 'upper':
        va = 'top'
        y = 1-offsety/axh
    elif v == 'lower':
        va = 'bottom'
        y = offsety/axh
    else:
        raise Exception('unknown location')

    if h == 'left':
        ha = 'left'
        x = offsetx/axw
    elif h == 'right':
        ha = 'right'
        x = 1-offsetx/axw
    else:
        raise Exception('unknown location')

    kw = dict(transform=ax.transAxes,
              verticalalignment=va,
              horizontalalignment=ha,
              size='x-large',
              zorder=-1)
    kw.update(kwargs)
    return ax.text(x, y, label, **kw)

Populating the interactive namespace from numpy and matplotlib


Preparation
-----------

We only look at packings which meet the following basic requirements:

1. $10^{-7} \leq P \leq 10^{-1.5}$
2. $|\sigma_{xy, \textrm{base}}| < 10^{-15}$ (packing equilibration)
3. $|\max(\nabla U)/P| < 10^{-8}$ (packing equilibration)

Then, during shear, they must also conform to the following two rules:

1. $N_\textrm{eff}^+ \leq N_\textrm{eff}^-$: events where rattlers becoming non-rattlers are ill-defined
2. $N^+ = 0$ and $N^- > 0$ _or_ $N^+ > 0$ and $N^- = 0$ (no composite events)

Last, but not least, we only include $(N,P)$ sets that have at least 20 conforming packings.


In [7]:
import tables
base = "E:/PhD rearrangements/auto/"
summ = tables.File(base + "h5/shear_summary_cache.h5").root.data.read()

dataN = summ["N"]
datalP = around(log10(summ["P"]),2)

#print "Total # of packings: ", len(summ)

# (1.1) pressure
mask = (datalP >= -7) * (datalP <= -1.5)
#print "After (1.1): ", len(summ[mask])

# (1.2) convergence initial packing
mask *= (abs(summ["sxy_base"]) < 1e-15)
#print "After (1.2): ", len(summ[mask])

# (1.3) convergence initial packing
mask *= (abs(summ["maxGrad_base"] / summ["P"]) < 1e-8)
#print "After (1.3): ", len(summ[mask])

# (2.1) no rattlers -> non-rattlers
mask *= (summ["Neff_plus"] <= summ["Neff_min"])
#print "After (2.1): ", len(summ[mask])

# (2.2) no composite events
mask *= logical_or( (summ["N+_plus"] == 0) * (summ["N-_plus"] >  0),
                    (summ["N+_plus"] >  0) * (summ["N-_plus"] == 0)
        )
#print "After (2.2): ", len(summ[mask])

# (3) only NP sets with more than 20 data points
#print "(3):"
for N in unique(dataN):
    for lP in unique(datalP):
        thisNPmask = mask * (dataN == N) * (datalP == lP)
        numpacks = len(summ[thisNPmask])
        if numpacks < 20:
            if numpacks > 0:
                #print "  ", N, lP, "skipped (only", numpacks, "packings)"
                mask[thisNPmask] = 0

#print "After (3): ", len(summ[mask])

summ = summ[mask]
dataN = summ["N"]
datalP = around(log10(summ["P"]),2)

In [5]:
import json
all_biax_measurements = json.load(open("u:/biax_metingen_shear.json"))

phi_norm = mpl.colors.Normalize(vmin=0.8, vmax=1.6)
phi_cmap = plt.get_cmap('viridis_r')

NameError: name 'np' is not defined

In [ ]:
import sys
sys.path.append("d:/my documents/phd-library")
sys.path.append(r"C:\Users\deen\Dropbox\PhD Rearrangements\figs")
from plotprint import printer

In [ ]:
from IPython.core.display import display_html, HTML

def hprint(h):
    display_html(HTML(h))

In [20]:
fig_height*2.5

8.186891748535585

0.75

In [8]:
#white = array([1,1,1])
red = array([1,0,0])
blue = array([0,0,1])

mnlN = log10(16)
mxlN = log10(4096)

#orange = array([1,.4,0])
#brown = array([139,69,19])/256.

#cmap = matplotlib
#cmap, norm = mpl.colors.from_levels_and_colors(np.arange(-0.5,7,1), colors)

import matplotlib
NPcmap = matplotlib.colors.LinearSegmentedColormap.from_list("BuPuRd", [blue, red])
NPnorm = matplotlib.colors.LogNorm(vmin=16, vmax=4096)

def NPmapper(N,P):
    rv = {'marker': {16: "o", 22: "v", 32: "D", 64: "^", 128: "p", 256: "*", 512: "s", 1024: "<", 2048: "d", 4096: ">"}.get(N, "o")}
    scl = (log10(N) - mnlN) / (mxlN - mnlN)
    
    if N <= 32:
        rv["marker"] = "<"
    elif N <= 512:
        rv["marker"] = "s"
    else:
        rv["marker"] = ">"

    rv["color"] = "white"
    
    rv["mec"] = NPcmap(NPnorm(N)) #blue + (red-blue) * scl
    return rv

In [6]:
if False:
    Ns = np.unique(dataN)

    fig = figure(figsize=(fig_width/2.05/4, fig_width/2.05))

    base_ax = fig.add_axes([
            0.05, # left
            half_fig_sppars.bottom, # bot
            0.5, # width
            half_fig_sppars.top - half_fig_sppars.bottom # heigth
    ] )

    base_ax.set_xticks([])
    base_ax.set_yticks([])

    ax = plt.twinx()
    for sp in ['top', 'right', 'bottom', 'left']:
        base_ax.spines[sp].set_visible(False)
        ax.spines[sp].set_visible(False)
        pass

    plt.axis(xmin=0, xmax=1, ymin=11.3, ymax=2896)

    plt.semilogy()
    ticks = [16,32,64,128,256,512,1024,2048]
    ax.set_yticks(ticks)
    ax.set_yticks([], minor=True)
    plt.setp(ax.get_yticklines(), visible=False)
    ax.set_yticklabels([str(x) for x in ticks])

    for pos, tick in zip(ticks, ax.yaxis.get_majorticklabels()):
        tick.set_x(0.25)
        tick.set_y(pos / 1.05)
        tick.set_y = lambda x: None

    plt.ylabel("N", rotation='horizontal')

    for N in Ns:
        kw  = NPmapper(N,1)
        kw.update(ms=2)
        #kw.update(**kwargs)
        plt.plot(0, N, clip_on=False,**kw)

    plt.savefig('general-legend.pdf')

In [3]:
try:
    if _oldsavefig:
        assert(_oldsavefig.__module__ != '__main__')
    else:
        raise Exception('dit kan niet')
except NameError:
    _oldsavefig = plt.savefig

from functools import wraps

def decorator(f):
    def _decorator(fn, *args, **kwargs):
        print r"\includegraphics{notebooks/%s/%s}" % (theNotebook, fn)

        p,ext = fn.rsplit(".")
        
        colorpng = p + "-color.png"
        bwpng = p + "-bw.png"
        print (r"\ifcolor\includegraphics{notebooks/%s/%s}" + 
        r"\else\includegraphics[width=%.3fin]{notebooks/%s/%s}\fi") % (
            theNotebook, fn, plt.gcf().get_size_inches()[0],
            theNotebook, bwpng
        )
        _oldsavefig(colorpng, dpi=300)
        from PIL import Image
        Image.open(colorpng).convert('L').save(bwpng)
        #colorfig = plt.imread(colorpng)
        #bwfig = 0.2989 * colorfig[:,:,0] + 0.5870 * colorfig[:,:,1] + 0.1140 * colorfig[:,:,2]
        #plt.imsave(bwpng, bwfig, cmap='gray', vmin=0, vmax=1)
        return f(fn, *args, **kwargs)
    _decorator.__doc__ = f.__doc__
    return _decorator

plt.savefig = decorator(_oldsavefig)
savefig = plt.savefig

NameError: name 'plt' is not defined